# Bayesian Basics for Classification Part 1

Throughout machine learning terms like probaility, a-posterior probability, likelihood and odds ratio crop up.  Actually understanding the interplayb between these related concepts can at first be difficult. Matters are not made simpler by the reality that in real-world applications information assumed by a rigorous application of these ideas lie beyond our reach.

For these reasons, this notebook creates a clean closed-world exampler were all the key elements are fully defined and this toy example is then useful as a means to clarify the interlocking pieces. 


Last Update 2/08/2022

**The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*


In [2]:
import numpy as np
import math as math
import matplotlib.pyplot as plt

# Part 1: The Binomial Distribution

The binomial distribution often takes center stage in situations where there are two possible outcomes and multiple independent samples are avaialble.  The common practical introduction to binomial distributions starts with a discussion of coin tossing.  

Consider tossing a fair coin 10 times. What is the probability of seeing heads exactly 5 times?

To begin, here is the probability mass function for the binomial distribution which gives us the exact answer to our question.

$$
P(k ; N, p) \; = \:
{{N}\choose{k}} p^{k} ( 1 - p )^{N-k}
$$

In this equation $N$ is the total number of trials, i.e. coin tosses. The value $k$ is the number of successes, i.e. times the coins comes up heads. The value of $p$ is the probability of success, i.e. probability the coin comes up heads on single toss. 

In [5]:
def n_choose_k (n,k) :
    res = math.factorial(n) // (math.factorial(k) * math.factorial(n-k))
    return res
    
def binomial_mass_function(k, n, p) :
    res = n_choose_k(n,k) * pow(p,k) * pow(1 - p, n - k)
    return res
    
foo = binomial_mass_function(1, 10000, 1/2)
foo

0.0

The answer to our question is that the probability of observing exactly $5$ heads in $10$ tosses for a fair coin is close to $25$ out of $100$ - or $0.25$

## Digression about Precision

The following may be a bit of overkill, but binomial distributions quickly raise interesting practical questions about probabilities and calculations. With that in mind, here is a variant on the standard form given above but in such a way that Python's ability to work with very large integers and true fractions helps us avoid floating point precision questions (entirely)

$$
P(k ; N, a, b) \; = \: \frac{{{N}\choose{k}}a^{k} ( b - a )^{N-k}}{b^{N}} 
\;\; \hbox{where} 
\;\; p = \frac{a}{b}
$$

In [7]:
from fractions import Fraction

def binomial_mass_function_rational(k, n, a, b) :
    num = n_choose_k(n,k) * pow(a,k) * pow(b - a, n - k)
    den = pow(b,n)
    return Fraction(num,den)
    
#rat = binomial_mass_function_rational(5, 10, 1, 2)
rat = binomial_mass_function_rational(5, 10, 1, 2)
print(f'The probability is {rat.numerator} / {rat.denominator} - as a float {rat_float:10.8f}' )

The probability is 52031268226562625 / 1246914448050473990552338851677240677389685519928870284282505593658089926888871664494997605252262162368708195588950570266972280709338361319849611620756020247365983127924612157922827909454128743470034380432387081971406788028788631542844317179130601555505655618427375578371352078615663848019342118909807024254076568540242152565422491976686590826429283635105894735976852547677305047545764283443227592261497273841185591493713177560033516738816538839208337127282847853095662696417514685587520995894790716206110829979110975385786376962999637386225461140355235142502331430342953243802154036340915659870600088427120100983411758703568461235569954862551684858479652757691376172364770343920520236829419398362679149564464258327587601512229480660878838728396395289925078053509206575464499516225737265633187053765106771300475370770010079280768031015990082052753846046184467944641019694443109094637707113762683227559849779047250650365365097879502810179188532191370039437761651

So far this may not to seem important, but in general realize that the probability of any particular outcome starts to become very small even for apparently simple problems.  Let us illustrate with a slight variant upon the above question.

What is the probability of seeing heads only once in $100$ tosses of a fair coin?

In [ ]:
rat = binomial_mass_function_rational(1, 100, 1, 2)
rat_float = float(rat)
print(f'The probability is {rat.numerator} / {rat.denominator} - as a float {rat_float:10.8f}' )

Take away from this little example that just because a probability of an outcome is terribly small do not confuse it with impossible.  

So for example, consider the following example where consider the set of all possible outcomes - by the way - this set of all possible outcomes it itself a KEY concept.

In [ ]:
foo = [binomial_mass_function_rational(i,100,1,2) for i in range(101)]
addem = foo[0]
for i in range(1,101) :
    addem = addem + foo[i]
addem

And while it may not seem like a big deal - here is the equivalent calcuation using our original setup in which probabilities were returned as floating point numbers.  Close, but not identical.

In [ ]:
foo = [binomial_mass_function(i,100,1/2) for i in range(101)]
addem = foo[0]
for i in range(1,101) :
    addem = addem + foo[i]
addem

# Overview: Reasoning from First Principals

We are about to do several things at once. 

From a very pragmatic side the next step is to go from our way of correctly computing the probability of $k$ heads in $n$ trials with a coin whose proability of heads on a given trial is $a$ divided by $b$.  

However, we doing this as a step along the path to working out optimal deicsion rules for problems where the underlying behavior is perfectly known. In other words, we will reason from first prinicipals based upon the assumption that our task involves fair (and unfair) coins. 

To the pragmatic part first, you may want to go review the [binomial distribution probability mass function](https://en.wikipedia.org/wiki/Binomial_distribution). It should then become clear the code snippets below show us (in plots) the probability of different outcomes when we toss a coin $k$ times. 

# Part 2: A Fair and an Unfair Coin 

Here are the probability mass functions for both a fair ($p=\frac{1}{2}$) and an unfair ($p=\frac{1}{5}$) coin. 

In [ ]:
tosses = 10; a = 1; b = 2
mass = np.array([float(binomial_mass_function_rational(i,tosses,a,b)) for i in range(tosses+1)])
np.set_printoptions(precision=4)
print(f'Probabilities for outcomes zero to n heads:\n {mass}')

fig = plt.figure(figsize=(8, 6))
plt.bar(np.arange(tosses+1),mass,color='darkgreen')
plt.xlabel('Number of Heads',fontsize=12);
plt.ylabel('Probability', fontsize=12);

What if the coins is not fair. In other words, it only comes up heads one out of five times?

In [ ]:
tosses = 10; a = 1; b = 5
mass = np.array([float(binomial_mass_function_rational(i,tosses,a,b)) for i in range(tosses+1)])
np.set_printoptions(precision=4)
print(f'Probabilities for outcomes zero to n heads:\n {mass}')

fig = plt.figure(figsize=(8, 6))
plt.bar(np.arange(tosses+1),mass,color='darkred')
plt.xlabel('Number of Heads',fontsize=12);
plt.ylabel('Probability', fontsize=12);

# Part 3: A Gambling Game

At the risk of gross oversimplification, machine learning is about using data to create strategies for intelligently guessing a label or a value. While perhaps a bit simplistic, this view quickly motivates some very important concepts which includ Bayes rule, Maximum A-Posteri estimation, Maximum Likelihood and finally odds ratios.  

However, let us start off with some sound commmon sense reasoning about the following game.

Here is how one round of the game is played. 

1. I am going to pick at random either a fair coin ($p = \frac{1}{2}$) or an unfair coind ($p = \frac{1}{5}$. I will then proceed to toss it $10$ times and we will both observe the number of times it comes up heads ($k$ observed). 

2. You must guess if the coin I picked at random is fair or unfair. If you guess correctly I give you a dollar, otherwise you give me a dollar.

We can play this game for as many rounds as you like - within reason and the limits of computer and simulation - and of course what matters is whether you can devise a strategy to win money.

## Simulating Game Play

In order to write the code to simulate game and different strategies we begin my taking advantage of the capabilities built into the NumPy random number generator.

In [ ]:
def tosses_heads (n, p) :
    rng = np.random.default_rng()
    cnt = np.sum(rng.random(10) > (1 - p) )
    return cnt

def trial_10_fair () : return tosses_heads(10, 1/2)

def trial_10_unfair () : return tosses_heads(10, 1/5)

Let us use the above to the outcomes of 100 games using only a fair coin.

In [ ]:
tosses = 10; n_games = 100
data = np.array([trial_10_fair() for i in range(n_games)])
sumc = np.bincount(data,minlength=tosses+1)
print(f'Game Outcomes:\n{data}')
print(f'Count of Outcomes:\n{sumc}')
print(f'Fewest and most heads seen: {np.min(data)} and {np.max(data)}')
None

Now let us look at the theoretical probability mass function - curve plot - and the actual empirical results over 1000 games. 

As it must/should be - the empirical result is close to what we would predict. 

To say a bit more, the comment above about reasoning from first principals is a broad statement of our approach and it is why we can be confident our empirical observations will fall in line with our prediction - we've constructed and artificial environment/task where that must be true.

### An Asside about Coding

You see below that the two plots are sufficiently similar that a function was written to generate each - thus highlighitng what is common and making changes to each simpler. As you coding skills develop you should aspire simplify and consolidate code in this manner. However, do not confuse the final product with how it came to be. Rest assured the instructor first wrote the brute force longer version and got it working - and only then did the more consolidated version you see here get written. 

In [ ]:
def gen_plot_theory_samples (ax, tosses, n_games, mass, freq, fair_p) :
    '''This function creates a plot where the empirical observation of number of heads 
    is shown in comparison to probability from first principals.
    The function is designed to readily generate similar plots 
    for fair and unfair coins.'''
    title_swap = ' Fair ' if fair_p else ' Unfair ' 
    base_color = 'darkgreen' if fair_p else 'darkred'
    ax.scatter(np.arange(tosses+1),mass,color=base_color,marker='_',s=300.0)
    ax.bar(np.arange(tosses+1),freq,color=base_color,alpha=0.5)
    ax.set_title("{:,}".format(n_games) + ' Games' + title_swap + 'Coin',fontsize=14)
    ax.set_xlabel('Number of Heads',fontsize=12);
    ax.set_ylabel('Probability / Frequency', fontsize=12);
    pass

In [ ]:
n_games = 100; tosses = 10

a = 1; b = 2
mass = np.array([float(binomial_mass_function_rational(i,tosses,a,b)) for i in range(tosses+1)])
data = np.array([trial_10_fair() for i in range(n_games)])
freq = np.bincount(data,minlength=tosses+1)/n_games

fig, ax = plt.subplots(1,2, figsize=(12, 6))
gen_plot_theory_samples(ax[0], tosses, n_games, mass, freq, True)

a = 1; b = 5
mass = np.array([float(binomial_mass_function_rational(i,tosses,a,b)) for i in range(tosses+1)])
data = np.array([trial_10_unfair() for i in range(n_games)])
freq = np.bincount(data,minlength=tosses+1)/n_games

gen_plot_theory_samples(ax[1], tosses, n_games, mass, freq, False)

# Play the Guessing Game

Now that we have the simulations we need it is time to build the game. 

## Player 1 - Always Fair

Just to get us started, let us build a player whose strategy is to simply always guess that the coin tossed was a fair coin. How do you expect this player to do in terms of winnings. To be clear, the player wins a dollar every time they guess correctly and loose a dollar everytime they guess incorrectly.

In [ ]:
def trial_10 (fair_p) :
    return tosses_heads(10, 1/2 if fair_p else 1/5)
    
def player_1 (n_heads) :
    '''Always choose fair coin which is signafied by returning True'''
    return True

def player_1_games (n) :
    rng = np.random.default_rng()
    fair = rng.random(n) > 1/2
    outcomes = np.array([trial_10(fp) for fp in fair])
    games = np.array([player_1(outcomes[i]) == fair[i] for i in range(n)])
    wins   = np.sum(games)
    losses = np.sum(~games)
    return wins - losses
    

In [ ]:
player_1_games(10**4)

## Player 2 - Most Likely

There is a very intuitively appealing strategy that leads nicely toward more formal decision theory. 

We'll get back to the more formal motivation shortly, but for now, let us just look at the probability mass functions for the fair and unfair coin.

In [ ]:
tosses = 10
mass_fair   = np.array([float(binomial_mass_function_rational(i,tosses,1,2)) for i in range(tosses+1)])
mass_unfair = np.array([float(binomial_mass_function_rational(i,tosses,1,5)) for i in range(tosses+1)])
np.set_printoptions(precision=4)

fig = plt.figure(figsize=(8, 6))
plt.bar(np.arange(tosses+1),mass_fair,color='darkgreen',alpha=0.6)
plt.bar(np.arange(tosses+1),mass_unfair,color='darkred',alpha=0.6)
plt.xlabel('Number of Heads',fontsize=12);
plt.ylabel('Probability', fontsize=12);

Just eye ball these to histograms and keep in mind you able to see each distribution even though the overlap because of semi-transparent rendering.  The fair coin probabilities are in green, the unfair in red, and since each is partially transparent where the distributions overlap you see brown.  

What should jump out is that the probability values are higher for an unfair versus a fair coin when $k$ is $0, 1, 2$ and $3$. Otherwise, the probability is higher for a fair coin.  

## Player 2

This clearly suggests a strategy; pick fair if and only if observed heads is 4 or greater.

In [ ]:
def player_2 (n_heads, threshold=3) :
    '''Choose fair coin over unfair iff number of heads observed is greater than 3'''
    if (n_heads > threshold) :
        return True
    else:
        return False

def player_2_games (n, print_accuracy=False) :
    '''This function will simulate n rounds of play for player 2. 
    The result is returned as the amount of money won by player 2. '''
    rng = np.random.default_rng()
    fair_true = rng.random(n) > 1/2
    outcomes  = np.array([trial_10(fp) for fp in fair_true])
    predicts  = np.array([player_2(outcomes[i]) for i in range(n)])
    corrects  = fair_true == predicts
    wins   = np.sum(corrects)
    losses = np.sum(~corrects)
    if print_accuracy : print(f'Accuracy of Guesses: {wins/n:5.3f}')
    return wins - losses
    

In [ ]:
player_2_games (10000)

What we've done so far should make a lot of intuitive sense; guess the most likely of the two cases, i.e. fair versus not fair, based upon the relative probability conditioned upon the number of heads actually observed. If you let player 2 play more games play 2 consistently earns more money.  This was not true for player 1.

### One More Thing - Accuracy

Notice that by introducing our guessing game as a gambling excercise we side-stepped ever actually measuring the accuracy of the guesses. Let us tidy up this small omission.

In [ ]:
player_2_games (10000, print_accuracy=True)

# Part 4: Accuracy Follows from First Principals

At this point we've shown that we can simulate game play and record and empirical estimate of how accurately Player 2 is guessing.  But because we have complete knowledge of the game we can of course go back to the original distributions and explain what we are oberving. 


In [ ]:
tosses = 10
threshold = 3
np.set_printoptions(precision=4)

mass_fair   = np.array([float(binomial_mass_function_rational(i,tosses,1,2)) for i in range(tosses+1)])
mass_unfair = np.array([float(binomial_mass_function_rational(i,tosses,1,5)) for i in range(tosses+1)])

# Note below we re-normalize so pair of distributions sum to 1.0
mass_fair = mass_fair / 2.0
mass_unfair = mass_unfair / 2.0

fair_mislabeled   = np.zeros(tosses+1)
unfair_mislabeled = np.zeros(tosses+1)
for i in range(0,threshold+1) : fair_mislabeled[i] = mass_fair[i]
for i in range(threshold+1,tosses+1) : unfair_mislabeled[i] = mass_unfair[i]

fig = plt.figure(figsize=(8, 6))
plt.bar(np.arange(tosses+1),mass_fair,color='darkgreen',alpha=0.6)
plt.bar(np.arange(tosses+1),mass_unfair,color='darkred',alpha=0.6)
plt.bar(np.arange(tosses+1),fair_mislabeled,color='orange',alpha=1.0)
plt.bar(np.arange(tosses+1),unfair_mislabeled,color='gold',alpha=1.0)

accuracy = 1.0 - np.sum(fair_mislabeled + unfair_mislabeled)
ptitle = f'Threshold is {threshold} and expected accuracy is {accuracy:5.3f}'
print(ptitle)
plt.title(ptitle,fontsize=14)
plt.xlabel('Number of Heads',fontsize=12);
plt.ylabel('Probability Conditioned Upon Class', fontsize=12);

Notice now that you can actually see very clearly the reason for selecting the threshold which will in expectation yield the most accurate predictions. To say a bit more, Player 2 guesses incorrectly when the fair coin generates as many heads (or fewer) than the theshold (shown in orange).  Player 2 also guesses incorrectly when the unfair coin generates more heads than the threshold (shown in gold).

One more important detail that foreshadows the next note book.  Note that the vertical axis probabilities are divided by two and are now described as conditioned upon whether the outcome is heads or tails. We will talk more explicitly next time about what to do when some classes arise more often in our sample than others.